# 1. Merge all files of sonhanha.com

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from xgboost import XGBRegressor
from xgboost import XGBClassifier
import pandas as pd

import selenium
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import datetime, timedelta
import os

import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [2]:
folder_path = "D:\OneDrive\KiotViet\Python_for_work\KFinance\CSV_crawl_data"
file_list = os.listdir(folder_path)

merged_df_new = pd.DataFrame()

# Loop qua từng file CSV và merge vào DataFrame
for file in file_list:
    if "cho_thue_cua_hang_kiot" in file or "sosanhnha_" in file:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        merged_df_new = pd.concat([merged_df_new, df], ignore_index=True)

# print(len(merged_df))
merged_df_new.drop_duplicates()
merged_df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15180 entries, 0 to 15179
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Title               14805 non-null  object 
 1   Price               14813 non-null  object 
 2   Area                14813 non-null  object 
 3   address             2813 non-null   object 
 4   type                15180 non-null  object 
 5   area_value          2813 non-null   float64
 6   price_value_trieu   2807 non-null   float64
 7   price_per_m2_final  2807 non-null   float64
 8   House_number        627 non-null    object 
 9   Street              2523 non-null   object 
 10  Ward                2649 non-null   object 
 11  District            2760 non-null   object 
 12  City                2771 non-null   object 
 13  progress            2813 non-null   object 
 14  cleaned_street      2813 non-null   object 
 15  Location            12000 non-null  object 
dtypes: f

In [3]:
merged_df_new = merged_df_new.dropna(axis=0)
merged_df_new.to_csv(r"D:\OneDrive\KiotViet\Python_for_work\KFinance\CSV_crawl_data\all_bds_sosanhnha.csv", index=False, mode = 'w', encoding="utf-8-sig")

# 2. Clean data 

In [4]:
df = pd.read_csv(r"D:\OneDrive\KiotViet\Python_for_work\KFinance\CSV_crawl_data\all_bds_sosanhnha.csv")
df.head()
print(f"The total rows of dataframe is {len(df)}")

The total rows of dataframe is 0


In [5]:
df = df[~df['Area'].str.contains('Đang cập nhật')]
df = df[~df['Price'].str.contains('Liên hệ')]
print(f"The total rows of dataframe is {len(df)}")

The total rows of dataframe is 0


2.0 Xử lý diện tích

In [6]:
# area_pattern = re.compile(r'(\d+) m²')
# df['area_value'] = df['Area'].apply(lambda x: int(area_pattern.search(x).group(1)) if area_pattern.search(x) else None)

df['area_value'] = df['Area'].apply(lambda x: float(x.replace('Diện tích: ', '').replace(' m²', ''))) #.replace('.', '')


In [7]:
df.head(1)

,Title,Price,Area,address,type,area_value,price_value_trieu,price_per_m2_final,House_number,Street,Ward,District,City,progress,cleaned_street,Location


2.1 Xử lý price

In [8]:
# Tạo cột 'price_per_m2'

#1.Nếu Price ghi X triệu/tháng và Area ghi Y m2 thì => Cột 'price_per_m2' thành x / Y
def convert_price_string_to_float(price_string):
    return float(price_string.replace(',', '.'))

price_pattern = re.compile(r'([\d,]+) (triệu/tháng|triệu)')

df['price_value_trieu'] = df['Price'].apply(lambda x: convert_price_string_to_float(price_pattern.search(x).group(1)) if price_pattern.search(x) else None)
df['price_per_m2_trieu'] = 1000000 * df['price_value_trieu'] / df['area_value']

#2.Nếu Price ghi X tỷ và Area ghi Y m2 thì => Cột 'price_per_m2' thành x / Y

price_pattern_bil = re.compile(r'([\d,]+) tỷ')

df['price_value_ty'] = df['Price'].apply(lambda x: convert_price_string_to_float(price_pattern_bil.search(x).group(1)) if price_pattern_bil.search(x) else None)
df['price_per_m2_ty'] = 1000000000 * df['price_value_ty'] / df['area_value']

df['price_value_trieu'] = df['price_value_trieu'].fillna(df['price_value_ty']*1000)


In [9]:
#4. Từ các cột "price_per_m2", "price_per_m2_trieu", "price_per_m2_ty" ta tìm được price_per_m2_final

def calculate_price_per_m2_final(row):
    if not pd.isna(row['price_per_m2_trieu']):
        return row['price_per_m2_trieu']
    elif not pd.isna(row['price_per_m2_ty']):
        return row['price_per_m2_ty']
    else:
        return np.nan

# Áp dụng hàm tùy chỉnh để tạo cột mới
df['price_per_m2_final'] = df.apply(calculate_price_per_m2_final, axis=1)
df = df.drop(columns=['price_per_m2_trieu','price_value_ty','price_per_m2_ty'])

2.2. Xử lý địa chỉ

In [10]:
df.rename(columns={'Location': 'address'}, inplace=True)
df.head(1)

,Title,Price,Area,address,type,area_value,price_value_trieu,price_per_m2_final,House_number,Street,Ward,District,City,progress,cleaned_street,address


In [11]:
list_api_keys = ["1EotnhALO_Xf7d5mlOVefECdfpbgPG_Dik0TeOcQauQ",
"92hxCAU523jWUFRXMhM-p3w66P3PxYfY_tZm9pPeShU",
"KwvOafjkobVhF8Iz63lMyNooS-hjaYgmHdZrqVVVHWw",
"N5qb2q5dSfly3fnqWMsDPqVMWUd1yRbTBxbT34hvw_g",
"SsQNhtNfemZJr_FzCYZlub29VeTTjymT3Qm6syC50W8",
"br4m9W3KMhrPa51Vru5M1mWJYCfaVFvWiP3kB2gEQUA",
"spdVYGb-b6cgokZBOuwGG4b3WbmgsxQqv7tudjdOfLc",
"0zeR-GEwo5Od-poIZYTGrw-I80fC2C0b0q4F1Boa8oc",
"UfHD68dH4O9ld-NS22Se6Bls98P9aPZhFq4hA_H7-74",
"c6NrKs_BpAX4OdRFKAVSDeUaQTjSpNF8dCxjcLT6AfE",
"WCpDyLiAACoL9zwSFQQa_HboP-HdWXSM601W1N02aeg",
"faJrm5egL12NmLivrDsbqz44I47DhbJQUIg89L3TdIo",
"cdXQHZ3zzsZv-n04kjimcml7e3BRKcR2H3wl8XPi6SU",
"RBRLSkMgpwy6QF0jKsXs-jXEliqtAwwD_jd0DbHrEN8",
"Dd-n66bTKOIKeQ0H6nRvIOMWatnJ_viPxKMNGpxsFa4",
"q-Ao_RRfZNyfQPp2XsnYGbrOItF6SlXN0jObPIKVnk",
"ufLjrWIrvjyLHzL1TZIEOZjKpgolDMBUPXUwoIblgs0",
"tvjh1g0Xygp6DMsH8n5hX41jgbNh3VE61MsnWHYkfZw",
"d_8_FhktHjeMh91gB9bBKGHgMJFC6auZqHThhlHL9Hw",
"P1RAwTjkXvZUQLjHyi06Zm1YP1KeaZMJdjaNN1ZeOeE",
"DQV3KQ2U9DLoro0jMQUoIr-PQMKBpKIEpOqXzF1A498",
"6IcCytBh4PkKOC3YE2Smw4IGdAyTD7XTtsmgtxF-kdY",
"PgxoArILmRDFQreVp3pC6bexYfcKIVwMMN4oKpa8VOo",
"mwA5L1GvWFvnwIYL-U1kP0RDsZ61ElOKAH7wnbsTs4M"]

In [12]:
# Hàm gọi api here để lấy thông tin vị trí từ địa chỉ chi tiết
import requests

def get_location(address, api_key ):
    #api_key = "mwA5L1GvWFvnwIYL-U1kP0RDsZ61ElOKAH7wnbsTs4M"

    url = f'https://geocode.search.hereapi.com/v1/geocode?q={address}&apiKey={api_key}'
    
    try:
        # Gửi yêu cầu API
        response = requests.get(url)
        
        # Nếu API key đã hết hạn, thử API key tiếp theo trong danh sách

        if response.status_code == 429:
            return {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "False" }
        else: 
            data = response.json()    
            # Xử lý kết quả và trả về
            if data == {'items': []}: 
                list_address = {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "True" }
            else: 
                result = data['items'][0]
                position = result['position']
                lat = position.get('lat', '')
                long = position.get('lng', '')
                address_info = result['address']
                city = address_info.get('county', '')
                district = address_info.get('city', '')
                ward = address_info.get('district', '')
                street = address_info.get('street', '')
                houseNumber = address_info.get('houseNumber', '')
                list_address = {"lat": lat, "lng": long, "House_number": houseNumber, "Street": street, "Ward": ward, "District": district, "City": city, "progress": "True" }
    except:
           # Nếu thử xảy ra lỗi trả ra tập rỗng và progess là False đánh dấu những case chưa xử lí
        list_address = {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "False" }
           
    return list_address

# Hàm xử lí dữ liệu đầu vào là dataframe
def out_put(df, api_key):
    house_numbers = []
    streets = []
    wards = []
    districts = []
    cities = []
    progresses = []
    
    for address in df["address"]:
        address_info = get_location(address,api_key)
        house_numbers.append(address_info["House_number"])
        streets.append(address_info["Street"])
        wards.append(address_info["Ward"])
        districts.append(address_info["District"])
        cities.append(address_info["City"])
        progresses.append(address_info["progress"])
    
    df["House_number"] = house_numbers
    df["Street"] = streets
    df["Ward"] = wards
    df["District"] = districts
    df["City"] = cities
    df["progress"] = progresses

    return df

In [13]:
df_test = df

In [14]:
# Calculate the number of groups
num_groups = len(df_test) // 1000

# Calculate the number of remaining rows
remaining_rows = len(df_test) % 1000

# Create a list to store the new dataframes
dfs = []

# Split the dataframe into groups of two rows each
for i in range(num_groups):
    start_index = i * 1000
    end_index = start_index + 1000
    df_group = df_test[start_index:end_index].copy()
    dfs.append(df_group)

# Add the remaining rows to the last dataframe
if remaining_rows > 0:
    last_df = df_test[-remaining_rows:].copy()
    dfs.append(last_df)

In [15]:
# tiến hành chạy multi thread
import threading
import time

final = []
def out_put_with_time(df, api_key):
    start_time = time.time()
    out_put(df, api_key)
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Time: {execution_time/60} phút")

for df , api_key in zip(dfs, list_api_keys):
    thread = threading.Thread(target=out_put_with_time, args=(df,api_key))
    thread.start()
    thread.join()
    final.append(df)

print("Done")

Done


In [16]:
merged_df = pd.concat(final, ignore_index=True)

#check case false
# merged_df[merged_df["progress"] == "False"]
merged_df.head(1)

ValueError: No objects to concatenate

In [ ]:
merged_df = merged_df[merged_df["progress"] != "False"]

In [ ]:
def extract_street_name(address):
    if pd.isna(address):  # Kiểm tra xem giá trị có phải là NaN không
        return None

    # Sử dụng biểu thức chính quy để tìm tên đường
    match = re.search(r'(?:Hẻm\s*\d+\s*)|(?:Ngõ\s*\S+\s*)|(?:Ngách\s*\S+\s*)', address)
    
    if match:
        return re.sub(r'(?:Hẻm\s*\d+\s*)|(?:Ngõ\s*\S+\s*)|(?:Ngách\s*\S+\s*)', '', address)
    else:
        return address

In [ ]:
def extract_location_prefix_preferred(address):
    if pd.isna(address):  # Kiểm tra xem giá trị có phải là NaN không
        return None

    # Tìm kiếm từ khóa "phường", "xã", "thị trấn", "thị xã" và lấy phần trước đó
    match_keyword = re.search(r'(.+?)(?i:phường|xã|thị trấn|thị xã)', address)
    
    if match_keyword:
        result = match_keyword.group(1).strip()
    else:
        # Nếu không tìm thấy từ khóa, tìm kiếm từ đầu tiên cho đến dấu ','
        match_first_part = re.search(r'([^,]+[a-zA-Z]+[^,]*)', address)
        
        if match_first_part:
            result = match_first_part.group(1).strip()
        else:
            result = None
    
    # Xóa dấu ',' nếu có
    if result is not None:
        result = result.replace(',', '').replace('.','').replace('/','')
    
    return result

In [ ]:
# merged_df['merged_df'] = merged_df['Street'].apply(extract_street_name)
# df_fix_address['Location_Prefix_Preferred'] = df_fix_address['address'].apply(extract_location_prefix_preferred)
# # df_fix_address.dropna()
# # df_fix_address.head()

In [ ]:
# def final_street(row):
#     if not pd.isna(row['Street_fix']):
#         return row['Street_fix']
#     else:
#         return row['Location_Prefix_Preferred']
# df_fix_address['Street_final'] = df_fix_address.apply(final_street,axis=1)
# df_fix_address.head(1)

In [ ]:
merged_df['price_value_trieu'] = merged_df['price_per_m2_final']*merged_df['area_value']/1000000


In [ ]:
df_final_sosanhnha = merged_df[['City','District','Ward','Street','address','area_value','price_value_trieu','price_per_m2_final']]
# df_final_sosanhnha.head()

In [ ]:
df_final_sosanhnha.to_csv(r"D:\OneDrive\KiotViet\Python_for_work\KFinance\CSV_crawl_data\final_bds_sosanhnha.csv", index=False,mode = 'w', encoding="utf-8-sig")